In [ ]:
#hide
import rfpfolio as rfp

# Welcome to rfpfolio

> Create portfolios with rebalancing, and measure performance.

This file will become your README and also the index of your documentation.

## Install

`pip install rfpfolio`

## How to use

Some code examples:

In [ ]:
tst_src = rfp.PriceSource('testdata/2017-Apr')

In [ ]:
price_data = tst_src.loadAllAdjustedPrices(['SPY', 'IEI', 'GLD'])
price_data.head()

,SPY,IEI,GLD
Date,,,
2017-04-03,220.896225,115.845558,119.459999
2017-04-10,218.369843,116.867851,122.599998
2017-04-17,220.323349,116.943314,122.309998
2017-04-24,223.601120,116.688652,120.769997
2017-05-01,225.122589,116.301956,117.010002
